<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/export-ee-data.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/giswqs/qgis-earthengine-examples/tree/master/Folium/export-ee-data.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td><td>
<a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/qgis-earthengine-examples/blob/master/Folium/export-ee-data.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td></table>

# Exporting Data from Google Earth Engine

This notebook demonstrates how to export data from Google Earth Engine using the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install).

**Contact:** Qiusheng Wu (<https://wetlands.io>)

**Table of Contents**
* [Install Earth Engine API](#install-ee-api)
* [Import Earth Engine API and authenticate](#import-api)
* [Export ee.Image](#export-image)
* [Export ee.ImageCollection](#export-image-collection)
* [Export vector data](#export-vector)
* [Export tabular data](#export-table)

## Install Earth Engine API <a class="anchor" id="install-ee-api"></a>

The Earth Engine API is installed by default in [Google Colaboratory](https://colab.sandbox.google.com/notebooks/welcome.ipynb). You only need to install it if you are running this notebook using your local Python interpreter or other cloud computing platforms.  

In [ ]:
!pip install earthengine-api

## Import Earth Engine API and authenticate<a class="anchor" id="import-api"></a>

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [2]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [3]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

### Test the API

Test the API by printing the elevation of Mount Everest.

In [4]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


`ee.Image` objects can be displayed to notebook output cells. The `IPython.display` module contains the `Image` function, which can display the results of a URL representing an image generated from a call to the Earth
Engine `getThumbUrl` function. The following cell will display a thumbnail of the global elevation model.

In [5]:
# Import the Image function from the IPython.display module. 
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbUrl({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

## Exporting ee.Image <a class="anchor" id="export-image"></a>

You can export images from Earth Engine in [GeoTIFF](https://trac.osgeo.org/geotiff/) or [TFRecord format](https://www.tensorflow.org/api_guides/python/python_io#TFRecords_Format_Details). See [Configuration Parameters](https://developers.google.com/earth-engine/exporting#configuration-parameters) for more output options.

To export an image to your Drive account, use `Export.image.toDrive()`. For example, to export portions of a Landsat image, define a region to export, then call `Export.image.toDrive()`:

In [6]:
import ee

image = ee.Image('USDA/NAIP/DOQQ/m_4609915_sw_14_1_20100629')
region = image.geometry()  # specify the roi 
scale = image.projection().nominalScale().multiply(10)  # specify the image resolution
description = image.get('system:index').getInfo()  # set the output image filename

# Set configration parameters for output image
task_config = {
    'folder': 'gee-data', # output Google Drive folder
    'region': region,     # roi 
    'scale': scale,       # image resolution
    'crs': 'EPSG:4326',
    'maxPixels': 1.0E13,
    'fileFormat': 'GeoTIFF'
    }

# Export image to Google Drive
task = ee.batch.Export.image.toDrive(image, description, **task_config)
task.start()
print("Exporting {}".format(description))

Exporting m_4609915_sw_14_1_20100629


## Exporting ee.ImageCollection  <a class="anchor" id="export-image-collection"></a>

To export images from an ee.ImageCollection, use a for loop. 

In [7]:
import ee

# Create an ImageCollection
loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(loc) \
    .filterDate('2008-01-01', '2020-01-01') \
    .filter(ee.Filter.listContains("system:band_names", "N"))

count = int(collection.size().getInfo())
print("Total number of images: {}".format(count))

for i in range(0, count):
    image = ee.Image(collection.toList(count).get(i))
    region = image.geometry() # roi
    scale = image.projection().nominalScale().multiply(10)  # specify the image resolution

    task_config = {
        'folder': 'gee-data', # output Google Drive folder
        'region': region,     # specify roi
        'scale': scale,       # specify image resolution
        'crs': 'EPSG:4326',
        'maxPixels': 1.0E13,
        'fileFormat': 'GeoTIFF'
    }
    
    description = image.get('system:index').getInfo()
    print('Exporting {}/{}: {}'.format(i+1, count, description))
    task = ee.batch.Export.image.toDrive(image, description, **task_config)
    task.start()

Total number of images: 6
Exporting 1/6: m_4609915_sw_14_1_20090818
Exporting 2/6: m_4609915_sw_14_1_20100629
Exporting 3/6: m_4609915_sw_14_1_20120714
Exporting 4/6: m_4609915_sw_14_1_20140901
Exporting 5/6: m_4609915_sw_14_1_20150926
Exporting 6/6: m_4609915_sw_14_h_20170703


## Exporting vector data  <a class="anchor" id="export-vector"></a>

You can export a `FeatureCollection` as CSV, SHP (shapefile), GeoJSON, KML, KMZ or TFRecord using `Export.table`. The `FeatureCollection` may represent vectors or simply a table of data. In the latter case, the features in the collection will have null geometry.

In [8]:
import ee

fc = ee.FeatureCollection('TIGER/2018/States') \
    .filter(ee.Filter.eq('STUSPS', 'TN'))

desc = 'vectorsToDriveExample'

# Set configration parameters for output vector
task_config = {
    'folder': 'gee-data', # output Google Drive folder
    'fileFormat': 'SHP',  # output formats: SHP, GeoJSON, KML, KMZ, TFRecord
    }

print('Exporting {}'.format(desc))
task = ee.batch.Export.table.toDrive(fc, desc, **task_config)
task.start()

Exporting vectorsToDriveExample


## Exporting tabular data  <a class="anchor" id="export-table"></a>

To export tabular data, you can set the `selectors` parameter to specify a list of properties/attributes to export. 

In [9]:
import ee

fc = ee.FeatureCollection('TIGER/2018/States') \
    .filter(ee.Filter.eq('STUSPS', 'TN'))

desc = 'tableToDriveExample'

# Set configration parameters for output vector
task_config = {
    'folder': 'gee-data', # output Google Drive folder
    'fileFormat': 'CSV',  
    'selectors': ['ALAND', 'NAME', 'STUSPS'] # a list of properties/attributes to be exported
    }

print('Exporting {}'.format(desc))
task = ee.batch.Export.table.toDrive(fc, desc, **task_config)
task.start()

Exporting tableToDriveExample
